In [10]:
from gensim.models import KeyedVectors
import numpy as np
import string
import csv
import re

model = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

#不要語削除
def chenge_text(text):
    text = text.translate(str.maketrans( '', '',string.punctuation))
    text = re.sub(r'[.]{2,}','.',text)
    text = re.sub(r'[	]',' ',text)
    text = re.sub(r'[ ]{2,}',' ',text)
    return text

def docvec(row):
    feature_vec = np.zeros((300,),dtype='float32') 
    words = row.split(" ")
    count = 0
    for word in words:
        try: 
            vec = model.wv[word]
            feature_vec += vec
            count += 1
        except:
            pass
    if count == 0:
        return feature_vec
    else:
        return feature_vec/count

with open('../data'+os.sep+'dbpedia'+os.sep+'dbpedia_csv'+os.sep+'classes.txt','r',encoding='utf-8') as f:
    classes = []
    reader = f.read().splitlines()
    for c in reader:
        classes.append(docvec(c))

with open('../data'+os.sep+'dbpedia'+os.sep+'dbpedia_csv'+os.sep+'test.csv','r',encoding='utf-8') as f:
    reader = csv.reader(f)
    x_test = []
    y_test = []
    for row in reader:
        text_stock = []
        text = row[2][1:].replace('(',')').split(')')
        for i,t in enumerate(text):
            if i % 2 == 0:
                text_stock.append(t)
        text = ''.join(text_stock)
        text = chenge_text(text)
        text = ' '.join([x for x in text.split(' ') if x not in row[1].split(' ')])
        text = text.replace('  ',' ')
        x_test.append(docvec(chenge_text(text)))
        y_test.append(int(row[0])-1)

np.save('../dataset/test/x_test.npy',x_test)
np.save('../dataset/test/y_test.npy',y_test)
np.save('../dataset/test/classes.npy',classes)

/home/dbl/.pyenv/versions/3.6.9/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [12]:
import numpy as np

def rank1(vec,class_list):
    min=-1
    pp=1000
    for j in range(len(class_list)):
        vec1 = vec
        vec2 = class_list[j]
        cos = np.sum((vec1*vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)))
        if cos>min:
            min=cos
            pp=j
    return pp

x_test = np.load('../dataset/test/x_test.npy')
y_test = np.load('../dataset/test/y_test.npy')
classes = np.load('../dataset/test/classes.npy')

pred = []
for i,vec in zip(y_test,x_test):
    pp = rank1(vec,classes)
    pred.append(pp)

from sklearn import metrics
rep = metrics.classification_report(y_test,pred,digits=3)
print(rep)

              precision    recall  f1-score   support

           0      0.854     0.416     0.560      5000
           1      0.501     0.755     0.602      5000
           2      0.861     0.331     0.479      5000
           3      0.968     0.645     0.774      5000
           4      0.432     0.349     0.386      5000
           5      0.423     0.473     0.447      5000
           6      0.611     0.474     0.534      5000
           7      0.201     0.496     0.286      5000
           8      0.643     0.984     0.778      5000
           9      0.754     0.064     0.118      5000
          10      0.615     0.485     0.542      5000
          11      0.785     0.931     0.852      5000
          12      0.767     0.739     0.753      5000
          13      0.509     0.624     0.561      5000
        1000      0.000     0.000     0.000         0

    accuracy                          0.555     70000
   macro avg      0.595     0.518     0.511     70000
weighted avg      0.637   